# ⚡ Notebook 3 : Optimisation & Évaluation des Prompts

## Métriques de Qualité, A/B Testing et Optimisation Automatique

Dans ce notebook, nous explorons les techniques scientifiques pour mesurer, évaluer et optimiser automatiquement la performance de vos prompts.

### 🎯 Objectifs d'apprentissage :
- Définir des métriques objectives de qualité des prompts
- Implémenter des systèmes d'A/B testing pour prompts
- Créer des pipelines d'optimisation automatique
- Mesurer ROI et impact business des améliorations
- Construire des dashboards de monitoring en temps réel

## 📚 Configuration et Imports

In [ ]:
import os
import json
import time
import uuid
import random
import sqlite3
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Any
from dataclasses import dataclass, asdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import accuracy_score, f1_score
from dotenv import load_dotenv

# Charger les variables d'environnement
load_dotenv()

# Configuration des APIs
import openai
from anthropic import Anthropic
import google.generativeai as genai

# Configuration des clients
openai.api_key = os.getenv('OPENAI_API_KEY')
anthropic = Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Configuration terminée !")

## 1. 📊 Métriques de Qualité des Prompts

### Framework de Métriques Comprehensive

Nous allons implémenter un système complet de métriques pour évaluer la qualité des prompts selon plusieurs dimensions :

- **Précision** : Justesse de la réponse
- **Complétude** : Exhaustivité du contenu
- **Cohérence** : Logique interne
- **Utilité** : Valeur actionnable
- **Efficiency** : Ratio qualité/coût

In [ ]:
@dataclass
class PromptMetrics:
    """Structure pour les métriques d'un prompt"""
    prompt_id: str
    prompt_text: str
    response: str
    timestamp: datetime
    
    # Métriques de qualité (0-1)
    accuracy: float = 0.0
    completeness: float = 0.0
    coherence: float = 0.0
    usefulness: float = 0.0
    clarity: float = 0.0
    
    # Métriques techniques
    response_time: float = 0.0
    token_count: int = 0
    cost_usd: float = 0.0
    
    # Métriques business
    user_satisfaction: Optional[float] = None
    task_completion: bool = False
    
    def overall_score(self) -> float:
        """Score global pondéré"""
        weights = {
            'accuracy': 0.25,
            'completeness': 0.20,
            'coherence': 0.20,
            'usefulness': 0.25,
            'clarity': 0.10
        }
        
        score = sum(getattr(self, metric) * weight 
                   for metric, weight in weights.items())
        return round(score, 3)
    
    def efficiency_score(self) -> float:
        """Ratio qualité/coût"""
        if self.cost_usd == 0:
            return 0
        return self.overall_score() / self.cost_usd

class PromptEvaluator:
    """Système d'évaluation automatique des prompts"""
    
    def __init__(self, model: str = "gpt-3.5-turbo"):
        self.model = model
        self.evaluations_db = "prompt_evaluations.db"
        self.init_database()
    
    def init_database(self):
        """Initialise la base de données pour stocker les évaluations"""
        conn = sqlite3.connect(self.evaluations_db)
        cursor = conn.cursor()
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS evaluations (
            id TEXT PRIMARY KEY,
            prompt_text TEXT,
            response TEXT,
            timestamp DATETIME,
            accuracy REAL,
            completeness REAL,
            coherence REAL,
            usefulness REAL,
            clarity REAL,
            response_time REAL,
            token_count INTEGER,
            cost_usd REAL,
            user_satisfaction REAL,
            task_completion BOOLEAN,
            overall_score REAL
        )
        """)
        
        conn.commit()
        conn.close()
    
    def evaluate_response_quality(self, prompt: str, response: str, context: str = "") -> Dict[str, float]:
        """Évalue la qualité d'une réponse selon plusieurs critères"""
        evaluation_prompt = f"""
        Évalue cette interaction prompt-réponse selon 5 critères précis :
        
        PROMPT : {prompt}
        {f'CONTEXTE : {context}' if context else ''}
        RÉPONSE : {response}
        
        Pour chaque critère, donne une note de 0 à 10 :
        
        1. ACCURACY (Précision) : La réponse est-elle factuelle et correcte ?
        2. COMPLETENESS (Complétude) : La réponse couvre-t-elle tous les aspects importants ?
        3. COHERENCE (Cohérence) : La logique interne est-elle solide ?
        4. USEFULNESS (Utilité) : La réponse est-elle actionnable et pratique ?
        5. CLARITY (Clarté) : La réponse est-elle claire et bien structurée ?
        
        Format de réponse OBLIGATOIRE :
        ACCURACY: X/10
        COMPLETENESS: Y/10
        COHERENCE: Z/10
        USEFULNESS: W/10
        CLARITY: V/10
        """
        
        try:
            result = openai.ChatCompletion.create(
                model=self.model,
                messages=[{"role": "user", "content": evaluation_prompt}],
                temperature=0.1
            )
            
            # Parser les scores
            content = result.choices[0].message.content
            scores = {}
            
            import re
            for criterion in ['ACCURACY', 'COMPLETENESS', 'COHERENCE', 'USEFULNESS', 'CLARITY']:
                pattern = f"{criterion}:\s*(\d+(?:\.\d+)?)/10"
                match = re.search(pattern, content, re.IGNORECASE)
                if match:
                    scores[criterion.lower()] = float(match.group(1)) / 10
                else:
                    scores[criterion.lower()] = 0.5  # Score par défaut
            
            return scores
            
        except Exception as e:
            print(f"Erreur évaluation : {e}")
            return {k: 0.5 for k in ['accuracy', 'completeness', 'coherence', 'usefulness', 'clarity']}
    
    def calculate_technical_metrics(self, response: str, start_time: float, model: str) -> Dict:
        """Calcule les métriques techniques"""
        end_time = time.time()
        response_time = end_time - start_time
        
        # Estimation du nombre de tokens (approximation)
        token_count = len(response.split()) * 1.3  # Approximation
        
        # Estimation du coût (tarifs approximatifs 2024)
        cost_per_1k_tokens = {
            'gpt-3.5-turbo': 0.002,
            'gpt-4': 0.03,
            'gpt-4-turbo': 0.01,
            'claude-3-sonnet': 0.003,
            'claude-3-opus': 0.015
        }
        
        cost_rate = cost_per_1k_tokens.get(model, 0.002)
        cost_usd = (token_count / 1000) * cost_rate
        
        return {
            'response_time': response_time,
            'token_count': int(token_count),
            'cost_usd': round(cost_usd, 4)
        }
    
    def evaluate_prompt_complete(self, prompt: str, context: str = "", 
                               model: str = None) -> PromptMetrics:
        """Évaluation complète d'un prompt"""
        if model is None:
            model = self.model
        
        prompt_id = str(uuid.uuid4())
        start_time = time.time()
        
        # Générer la réponse
        try:
            response_obj = openai.ChatCompletion.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                temperature=0.7
            )
            response = response_obj.choices[0].message.content
        except Exception as e:
            print(f"Erreur génération : {e}")
            response = "Erreur lors de la génération"
        
        # Évaluer la qualité
        quality_scores = self.evaluate_response_quality(prompt, response, context)
        
        # Calculer les métriques techniques
        tech_metrics = self.calculate_technical_metrics(response, start_time, model)
        
        # Créer l'objet métriques
        metrics = PromptMetrics(
            prompt_id=prompt_id,
            prompt_text=prompt,
            response=response,
            timestamp=datetime.now(),
            **quality_scores,
            **tech_metrics
        )
        
        # Sauvegarder en base
        self.save_evaluation(metrics)
        
        return metrics
    
    def save_evaluation(self, metrics: PromptMetrics):
        """Sauvegarde une évaluation en base de données"""
        conn = sqlite3.connect(self.evaluations_db)
        cursor = conn.cursor()
        
        cursor.execute("""
        INSERT INTO evaluations (
            id, prompt_text, response, timestamp, accuracy, completeness,
            coherence, usefulness, clarity, response_time, token_count,
            cost_usd, user_satisfaction, task_completion, overall_score
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            metrics.prompt_id, metrics.prompt_text, metrics.response,
            metrics.timestamp, metrics.accuracy, metrics.completeness,
            metrics.coherence, metrics.usefulness, metrics.clarity,
            metrics.response_time, metrics.token_count, metrics.cost_usd,
            metrics.user_satisfaction, metrics.task_completion, 
            metrics.overall_score()
        ))
        
        conn.commit()
        conn.close()
    
    def get_evaluation_history(self, limit: int = 100) -> pd.DataFrame:
        """Récupère l'historique des évaluations"""
        conn = sqlite3.connect(self.evaluations_db)
        
        query = """
        SELECT * FROM evaluations 
        ORDER BY timestamp DESC 
        LIMIT ?
        """
        
        df = pd.read_sql_query(query, conn, params=(limit,))
        conn.close()
        
        return df

### 🎯 Exemple : Évaluation de prompts différents

In [ ]:
# Initialiser l'évaluateur
evaluator = PromptEvaluator()

# Tester différents prompts pour la même tâche
task_context = "Création d'une stratégie marketing pour une startup EdTech"

prompts_to_test = [
    {
        'name': 'Prompt Basic',
        'text': "Crée une stratégie marketing pour notre startup EdTech."
    },
    {
        'name': 'Prompt Structuré',
        'text': """
        En tant qu'expert marketing spécialisé EdTech :
        
        Crée une stratégie marketing pour notre startup qui propose des cours de code en ligne.
        
        Inclus :
        - Analyse du marché cible
        - 3 canaux d'acquisition prioritaires
        - Budget et timeline sur 6 mois
        - Métriques de succès
        """
    },
    {
        'name': 'Prompt Avec Contexte',
        'text': """
        CONTEXTE : Startup EdTech française, 6 mois d'existence, 500 utilisateurs, 
        budget marketing 50k€, équipe de 3 personnes.
        
        RÔLE : Tu es un Growth Hacker expérimenté spécialisé dans l'EdTech B2C.
        
        TÂCHE : Conçois une stratégie d'acquisition client pour doubler notre base 
        utilisateur en 4 mois tout en maintenant un CAC < 100€.
        
        FORMAT : Plan actionnable avec timeline, budget détaillé et KPIs précis.
        """
    }
]

# Évaluer chaque prompt
results = []
print("🔬 Évaluation des prompts...\n")

for i, prompt_data in enumerate(prompts_to_test):
    print(f"{i+1}. Évaluation : {prompt_data['name']}")
    
    metrics = evaluator.evaluate_prompt_complete(
        prompt_data['text'], 
        task_context
    )
    
    results.append({
        'name': prompt_data['name'],
        'metrics': metrics
    })
    
    print(f"   Score global : {metrics.overall_score():.3f}")
    print(f"   Coût : ${metrics.cost_usd:.4f}")
    print(f"   Temps : {metrics.response_time:.2f}s\n")

# Comparaison des résultats
print("📊 COMPARAISON DES PROMPTS :")
print("=" * 50)

for result in sorted(results, key=lambda x: x['metrics'].overall_score(), reverse=True):
    m = result['metrics']
    print(f"\n🏆 {result['name']}")
    print(f"   Score global : {m.overall_score():.3f}")
    print(f"   Précision : {m.accuracy:.2f} | Complétude : {m.completeness:.2f}")
    print(f"   Cohérence : {m.coherence:.2f} | Utilité : {m.usefulness:.2f}")
    print(f"   Clarté : {m.clarity:.2f}")
    print(f"   Efficience : {m.efficiency_score():.1f} (qualité/coût)")

## 2. 🧪 A/B Testing pour Prompts

### Framework d'A/B Testing Statistiquement Robuste

Implémentons un système d'A/B testing pour comparer objectivement différentes versions de prompts avec une rigueur statistique.

In [ ]:
@dataclass
class ABTestConfig:
    """Configuration pour un test A/B"""
    test_name: str
    prompt_a: str
    prompt_b: str
    sample_size: int = 50
    significance_level: float = 0.05
    primary_metric: str = 'overall_score'
    secondary_metrics: List[str] = None
    test_contexts: List[str] = None
    
    def __post_init__(self):
        if self.secondary_metrics is None:
            self.secondary_metrics = ['accuracy', 'usefulness', 'efficiency_score']
        if self.test_contexts is None:
            self.test_contexts = [""]

@dataclass
class ABTestResult:
    """Résultats d'un test A/B"""
    test_name: str
    sample_size_a: int
    sample_size_b: int
    
    # Statistiques descriptives
    mean_a: float
    mean_b: float
    std_a: float
    std_b: float
    
    # Tests statistiques
    p_value: float
    is_significant: bool
    confidence_interval: Tuple[float, float]
    effect_size: float  # Cohen's d
    
    # Métrics business
    lift_percentage: float
    winner: str  # 'A', 'B', ou 'No significant difference'
    power: float  # Puissance statistique

class PromptABTester:
    """Système d'A/B testing pour prompts"""
    
    def __init__(self, evaluator: PromptEvaluator):
        self.evaluator = evaluator
        self.tests_db = "ab_tests.db"
        self.init_tests_database()
    
    def init_tests_database(self):
        """Initialise la base de données pour les tests A/B"""
        conn = sqlite3.connect(self.tests_db)
        cursor = conn.cursor()
        
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS ab_tests (
            test_id TEXT,
            test_name TEXT,
            variant TEXT,
            prompt_text TEXT,
            context TEXT,
            response TEXT,
            overall_score REAL,
            accuracy REAL,
            usefulness REAL,
            efficiency_score REAL,
            timestamp DATETIME,
            PRIMARY KEY (test_id, variant)
        )
        """)
        
        conn.commit()
        conn.close()
    
    def run_ab_test(self, config: ABTestConfig) -> ABTestResult:
        """Exécute un test A/B complet"""
        print(f"🧪 Démarrage du test A/B : {config.test_name}")
        print(f"📊 Taille d'échantillon : {config.sample_size} par variant")
        
        test_id = str(uuid.uuid4())
        results_a = []
        results_b = []
        
        # Générer les échantillons pour chaque variant
        print("\n🔄 Génération des échantillons...")
        
        # Utiliser ThreadPoolExecutor pour paralléliser
        with ThreadPoolExecutor(max_workers=4) as executor:
            # Soumettre les tâches pour variant A
            futures_a = []
            for i in range(config.sample_size):
                context = random.choice(config.test_contexts)
                future = executor.submit(self._evaluate_variant, 
                                       config.prompt_a, context, 'A', test_id, config.test_name)
                futures_a.append(future)
            
            # Soumettre les tâches pour variant B
            futures_b = []
            for i in range(config.sample_size):
                context = random.choice(config.test_contexts)
                future = executor.submit(self._evaluate_variant, 
                                       config.prompt_b, context, 'B', test_id, config.test_name)
                futures_b.append(future)
            
            # Collecter les résultats A
            for i, future in enumerate(as_completed(futures_a)):
                try:
                    result = future.result(timeout=60)
                    results_a.append(result)
                    if (i + 1) % 10 == 0:
                        print(f"   Variant A : {i + 1}/{config.sample_size} terminés")
                except Exception as e:
                    print(f"   Erreur variant A : {e}")
            
            # Collecter les résultats B
            for i, future in enumerate(as_completed(futures_b)):
                try:
                    result = future.result(timeout=60)
                    results_b.append(result)
                    if (i + 1) % 10 == 0:
                        print(f"   Variant B : {i + 1}/{config.sample_size} terminés")
                except Exception as e:
                    print(f"   Erreur variant B : {e}")
        
        print(f"✅ Collecte terminée : {len(results_a)} vs {len(results_b)} échantillons")
        
        # Analyser les résultats
        return self._analyze_results(config, results_a, results_b)
    
    def _evaluate_variant(self, prompt: str, context: str, variant: str, 
                         test_id: str, test_name: str) -> Dict:
        """Évalue un variant et sauvegarde en base"""
        metrics = self.evaluator.evaluate_prompt_complete(prompt, context)
        
        # Sauvegarder en base
        conn = sqlite3.connect(self.tests_db)
        cursor = conn.cursor()
        
        cursor.execute("""
        INSERT INTO ab_tests (
            test_id, test_name, variant, prompt_text, context, response,
            overall_score, accuracy, usefulness, efficiency_score, timestamp
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            test_id, test_name, variant, prompt, context, metrics.response,
            metrics.overall_score(), metrics.accuracy, metrics.usefulness,
            metrics.efficiency_score(), datetime.now()
        ))
        
        conn.commit()
        conn.close()
        
        return {
            'overall_score': metrics.overall_score(),
            'accuracy': metrics.accuracy,
            'usefulness': metrics.usefulness,
            'efficiency_score': metrics.efficiency_score()
        }
    
    def _analyze_results(self, config: ABTestConfig, 
                        results_a: List[Dict], results_b: List[Dict]) -> ABTestResult:
        """Analyse statistique des résultats A/B"""
        print("\n📊 Analyse statistique...")
        
        # Extraire la métrique principale
        values_a = [r[config.primary_metric] for r in results_a]
        values_b = [r[config.primary_metric] for r in results_b]
        
        # Statistiques descriptives
        mean_a, mean_b = np.mean(values_a), np.mean(values_b)
        std_a, std_b = np.std(values_a, ddof=1), np.std(values_b, ddof=1)
        
        # Test t de Student
        t_stat, p_value = stats.ttest_ind(values_a, values_b, equal_var=False)
        
        # Significance
        is_significant = p_value < config.significance_level
        
        # Intervalle de confiance pour la différence
        diff = mean_b - mean_a
        se_diff = np.sqrt((std_a**2 / len(values_a)) + (std_b**2 / len(values_b)))
        margin_error = stats.t.ppf(1 - config.significance_level/2, 
                                  len(values_a) + len(values_b) - 2) * se_diff
        ci = (diff - margin_error, diff + margin_error)
        
        # Effect size (Cohen's d)
        pooled_std = np.sqrt(((len(values_a)-1)*std_a**2 + (len(values_b)-1)*std_b**2) / 
                           (len(values_a) + len(values_b) - 2))
        cohens_d = (mean_b - mean_a) / pooled_std if pooled_std > 0 else 0
        
        # Lift percentage
        lift = ((mean_b - mean_a) / mean_a * 100) if mean_a > 0 else 0
        
        # Déterminer le gagnant
        if not is_significant:
            winner = "No significant difference"
        else:
            winner = "B" if mean_b > mean_a else "A"
        
        # Calcul de la puissance (approximation)
        effect_size_for_power = abs(cohens_d)
        power = self._calculate_power(len(values_a), effect_size_for_power, config.significance_level)
        
        return ABTestResult(
            test_name=config.test_name,
            sample_size_a=len(values_a),
            sample_size_b=len(values_b),
            mean_a=mean_a,
            mean_b=mean_b,
            std_a=std_a,
            std_b=std_b,
            p_value=p_value,
            is_significant=is_significant,
            confidence_interval=ci,
            effect_size=cohens_d,
            lift_percentage=lift,
            winner=winner,
            power=power
        )
    
    def _calculate_power(self, sample_size: int, effect_size: float, alpha: float) -> float:
        """Calcule la puissance statistique (approximation)"""
        # Calcul simplifié de la puissance
        from scipy.stats import norm
        
        z_alpha = norm.ppf(1 - alpha/2)
        z_beta = effect_size * np.sqrt(sample_size/2) - z_alpha
        power = norm.cdf(z_beta)
        
        return max(0, min(1, power))
    
    def visualize_results(self, result: ABTestResult, save_path: str = None):
        """Visualise les résultats du test A/B"""
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. Comparaison des moyennes
        variants = ['Variant A', 'Variant B']
        means = [result.mean_a, result.mean_b]
        stds = [result.std_a, result.std_b]
        
        bars = ax1.bar(variants, means, yerr=stds, capsize=10, 
                      color=['lightblue', 'lightcoral'], alpha=0.7)
        ax1.set_title(f'{result.test_name}\nComparaison des Performances')
        ax1.set_ylabel('Score Moyen')
        
        # Annoter avec les valeurs
        for bar, mean in zip(bars, means):
            ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                    f'{mean:.3f}', ha='center', va='bottom')
        
        # 2. Distribution des scores
        # Simuler les distributions pour visualisation
        samples_a = np.random.normal(result.mean_a, result.std_a, 1000)
        samples_b = np.random.normal(result.mean_b, result.std_b, 1000)
        
        ax2.hist(samples_a, bins=30, alpha=0.7, label='Variant A', color='lightblue')
        ax2.hist(samples_b, bins=30, alpha=0.7, label='Variant B', color='lightcoral')
        ax2.set_title('Distribution des Scores')
        ax2.set_xlabel('Score')
        ax2.set_ylabel('Fréquence')
        ax2.legend()
        
        # 3. Intervalle de confiance
        diff = result.mean_b - result.mean_a
        ci_lower, ci_upper = result.confidence_interval
        
        ax3.errorbar([0], [diff], yerr=[[diff - ci_lower], [ci_upper - diff]], 
                    fmt='o', markersize=10, capsize=10, capthick=2)
        ax3.axhline(y=0, color='red', linestyle='--', alpha=0.7)
        ax3.set_title('Différence entre Variants\n(avec Intervalle de Confiance 95%)')
        ax3.set_ylabel('Différence (B - A)')
        ax3.set_xlim(-0.5, 0.5)
        ax3.set_xticks([])
        
        # 4. Résumé statistique
        ax4.axis('off')
        summary_text = f"""
        📊 RÉSULTATS DU TEST A/B
        
        🏆 Gagnant : {result.winner}
        📈 Lift : {result.lift_percentage:+.1f}%
        
        📋 Statistiques :
        • p-value : {result.p_value:.4f}
        • Significatif : {'✅ Oui' if result.is_significant else '❌ Non'}
        • Effect Size : {result.effect_size:.3f}
        • Puissance : {result.power:.1%}
        
        📏 Échantillons :
        • Variant A : {result.sample_size_a}
        • Variant B : {result.sample_size_b}
        """
        
        ax4.text(0.1, 0.9, summary_text, transform=ax4.transAxes, 
                fontsize=11, verticalalignment='top', fontfamily='monospace')
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        plt.show()
        
        return fig

### 🎯 Exemple : Test A/B sur des prompts de résumé

In [ ]:
# Configuration du test A/B
ab_config = ABTestConfig(
    test_name="Résumé d'Articles - Prompt Simple vs Structuré",
    
    prompt_a="""Résume cet article en 3 points principaux.""",
    
    prompt_b="""En tant qu'analyste expérimenté :
    
    Analyse cet article et produis un résumé structuré :
    
    🎯 IDÉE PRINCIPALE (1 phrase)
    📋 POINTS CLÉS (3 éléments maximum)
    💡 INSIGHTS ACTIONABLES (1-2 recommandations)
    
    Reste factuel et concis.""",
    
    sample_size=20,  # Réduire pour la démo
    primary_metric='overall_score',
    
    test_contexts=[
        "Article tech sur l'IA",
        "Article business sur les startups", 
        "Article scientifique sur le climat",
        "Article économique sur les marchés"
    ]
)

# Exécuter le test A/B
ab_tester = PromptABTester(evaluator)
test_result = ab_tester.run_ab_test(ab_config)

# Afficher les résultats
print("\n" + "="*60)
print("🧪 RÉSULTATS DU TEST A/B")
print("="*60)

print(f"\n📊 Test : {test_result.test_name}")
print(f"\n🏆 Gagnant : {test_result.winner}")
print(f"📈 Amélioration : {test_result.lift_percentage:+.1f}%")

print(f"\n📋 Statistiques détaillées :")
print(f"  Variant A - Moyenne : {test_result.mean_a:.3f} (±{test_result.std_a:.3f})")
print(f"  Variant B - Moyenne : {test_result.mean_b:.3f} (±{test_result.std_b:.3f})")

print(f"\n🔬 Tests statistiques :")
print(f"  p-value : {test_result.p_value:.4f}")
print(f"  Significatif : {'✅ Oui' if test_result.is_significant else '❌ Non (p > 0.05)'}")
print(f"  Effect Size : {test_result.effect_size:.3f}")
print(f"  Puissance : {test_result.power:.1%}")

print(f"\n📏 Intervalles de confiance (95%) :")
ci_lower, ci_upper = test_result.confidence_interval
print(f"  Différence B-A : [{ci_lower:.3f}, {ci_upper:.3f}]")

# Visualiser les résultats
ab_tester.visualize_results(test_result)

## 3. 🤖 Optimisation Automatique des Prompts

### Framework d'Optimisation par Algorithme Génétique

Créons un système qui optimise automatiquement les prompts en utilisant des techniques d'optimisation inspirées de l'évolution naturelle.

In [ ]:
@dataclass
class PromptGene:
    """Représente un 'gène' dans un prompt (un élément modifiable)"""
    name: str
    type: str  # 'role', 'instruction', 'format', 'example', 'constraint'
    variations: List[str]
    current_value: str = ""
    
    def __post_init__(self):
        if not self.current_value and self.variations:
            self.current_value = random.choice(self.variations)
    
    def mutate(self) -> str:
        """Mute le gène en choisissant une nouvelle variation"""
        self.current_value = random.choice(self.variations)
        return self.current_value

@dataclass
class PromptChromosome:
    """Représente un prompt complet (chromosme) avec ses gènes"""
    genes: List[PromptGene]
    template: str
    fitness: float = 0.0
    generation: int = 0
    
    def generate_prompt(self) -> str:
        """Génère le prompt complet à partir des gènes"""
        prompt = self.template
        
        for gene in self.genes:
            placeholder = f"{{{gene.name}}}"
            prompt = prompt.replace(placeholder, gene.current_value)
        
        return prompt
    
    def mutate(self, mutation_rate: float = 0.3):
        """Applique des mutations aléatoires"""
        for gene in self.genes:
            if random.random() < mutation_rate:
                gene.mutate()
    
    def crossover(self, other: 'PromptChromosome') -> 'PromptChromosome':
        """Crée un offspring par croisement avec un autre chromosome"""
        new_genes = []
        
        for i, (gene1, gene2) in enumerate(zip(self.genes, other.genes)):
            # Choisir aléatoirement un parent pour chaque gène
            if random.random() < 0.5:
                new_gene = PromptGene(
                    name=gene1.name,
                    type=gene1.type,
                    variations=gene1.variations,
                    current_value=gene1.current_value
                )
            else:
                new_gene = PromptGene(
                    name=gene2.name,
                    type=gene2.type,
                    variations=gene2.variations,
                    current_value=gene2.current_value
                )
            new_genes.append(new_gene)
        
        return PromptChromosome(
            genes=new_genes,
            template=self.template,
            generation=max(self.generation, other.generation) + 1
        )

class GeneticPromptOptimizer:
    """Optimiseur de prompts par algorithme génétique"""
    
    def __init__(self, evaluator: PromptEvaluator):
        self.evaluator = evaluator
        self.population = []
        self.generation = 0
        self.best_fitness_history = []
        self.avg_fitness_history = []
    
    def create_initial_population(self, template: str, genes: List[PromptGene], 
                                population_size: int = 20) -> List[PromptChromosome]:
        """Crée la population initiale"""
        population = []
        
        for _ in range(population_size):
            # Créer des copies des gènes avec des valeurs aléatoires
            chromosome_genes = []
            for gene in genes:
                new_gene = PromptGene(
                    name=gene.name,
                    type=gene.type,
                    variations=gene.variations.copy(),
                    current_value=random.choice(gene.variations)
                )
                chromosome_genes.append(new_gene)
            
            chromosome = PromptChromosome(
                genes=chromosome_genes,
                template=template,
                generation=0
            )
            population.append(chromosome)
        
        return population
    
    def evaluate_fitness(self, chromosome: PromptChromosome, 
                        test_contexts: List[str], samples_per_context: int = 3) -> float:
        """Évalue la fitness d'un chromosome"""
        prompt = chromosome.generate_prompt()
        total_score = 0
        total_samples = 0
        
        for context in test_contexts:
            for _ in range(samples_per_context):
                try:
                    metrics = self.evaluator.evaluate_prompt_complete(prompt, context)
                    total_score += metrics.overall_score()
                    total_samples += 1
                except Exception as e:
                    print(f"Erreur évaluation : {e}")
                    # Pénalité pour les prompts qui causent des erreurs
                    total_score += 0.1
                    total_samples += 1
        
        fitness = total_score / total_samples if total_samples > 0 else 0
        chromosome.fitness = fitness
        return fitness
    
    def selection(self, population: List[PromptChromosome], 
                 selection_size: int) -> List[PromptChromosome]:
        """Sélection par tournoi"""
        selected = []
        
        for _ in range(selection_size):
            # Tournoi de taille 3
            tournament = random.sample(population, min(3, len(population)))
            winner = max(tournament, key=lambda x: x.fitness)
            selected.append(winner)
        
        return selected
    
    def evolve_generation(self, population: List[PromptChromosome],
                         elite_size: int = 2, mutation_rate: float = 0.3) -> List[PromptChromosome]:
        """Évolution d'une génération"""
        # Trier par fitness
        population.sort(key=lambda x: x.fitness, reverse=True)
        
        # Élitisme : garder les meilleurs
        new_population = population[:elite_size]
        
        # Sélection pour reproduction
        parents = self.selection(population, len(population) - elite_size)
        
        # Reproduction et mutation
        while len(new_population) < len(population):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            
            # Croisement
            child = parent1.crossover(parent2)
            
            # Mutation
            child.mutate(mutation_rate)
            
            new_population.append(child)
        
        return new_population
    
    def optimize(self, template: str, genes: List[PromptGene], 
                test_contexts: List[str], generations: int = 10,
                population_size: int = 20, mutation_rate: float = 0.3) -> Dict:
        """Lance l'optimisation complète"""
        print(f"🧬 Démarrage optimisation génétique")
        print(f"📊 Paramètres : {generations} générations, {population_size} individus")
        
        # Créer population initiale
        print("\n🌱 Création de la population initiale...")
        self.population = self.create_initial_population(template, genes, population_size)
        
        # Évoluer sur plusieurs générations
        for gen in range(generations):
            print(f"\n🔄 Génération {gen + 1}/{generations}")
            
            # Évaluer la fitness de chaque individu
            print("   📊 Évaluation de la fitness...")
            with ThreadPoolExecutor(max_workers=3) as executor:
                futures = []
                for chromosome in self.population:
                    future = executor.submit(self.evaluate_fitness, chromosome, 
                                           test_contexts, 2)  # 2 échantillons par contexte
                    futures.append(future)
                
                for future in as_completed(futures):
                    try:
                        future.result(timeout=120)
                    except Exception as e:
                        print(f"      Erreur : {e}")
            
            # Statistiques de la génération
            fitnesses = [c.fitness for c in self.population]
            best_fitness = max(fitnesses)
            avg_fitness = np.mean(fitnesses)
            
            self.best_fitness_history.append(best_fitness)
            self.avg_fitness_history.append(avg_fitness)
            
            print(f"   🏆 Meilleure fitness : {best_fitness:.3f}")
            print(f"   📈 Fitness moyenne : {avg_fitness:.3f}")
            
            # Évolution (sauf pour la dernière génération)
            if gen < generations - 1:
                print("   🧬 Évolution...")
                self.population = self.evolve_generation(self.population, 
                                                        elite_size=2, 
                                                        mutation_rate=mutation_rate)
        
        # Trouver le meilleur individu final
        best_chromosome = max(self.population, key=lambda x: x.fitness)
        
        return {
            'best_chromosome': best_chromosome,
            'best_prompt': best_chromosome.generate_prompt(),
            'best_fitness': best_chromosome.fitness,
            'fitness_history': {
                'best': self.best_fitness_history,
                'average': self.avg_fitness_history
            },
            'final_population': self.population
        }
    
    def visualize_evolution(self, save_path: str = None):
        """Visualise l'évolution de la fitness"""
        plt.figure(figsize=(12, 6))
        
        generations = range(1, len(self.best_fitness_history) + 1)
        
        plt.plot(generations, self.best_fitness_history, 
                'o-', label='Meilleure Fitness', linewidth=2, markersize=6)
        plt.plot(generations, self.avg_fitness_history, 
                's-', label='Fitness Moyenne', linewidth=2, markersize=4, alpha=0.7)
        
        plt.title('Évolution de la Fitness par Génération', fontsize=14, fontweight='bold')
        plt.xlabel('Génération')
        plt.ylabel('Score de Fitness')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # Annotations
        best_gen = np.argmax(self.best_fitness_history)
        best_score = self.best_fitness_history[best_gen]
        plt.annotate(f'Max: {best_score:.3f}', 
                    xy=(best_gen + 1, best_score), 
                    xytext=(10, 10), textcoords='offset points',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        plt.show()

### 🎯 Exemple : Optimisation d'un prompt de recommandation produit

In [ ]:
# Définir le template et les gènes pour l'optimisation
template = """
{role}

{task_instruction}

{context_guide}

{output_format}

{quality_constraint}
"""

# Définir les gènes (éléments variables du prompt)
optimization_genes = [
    PromptGene(
        name="role",
        type="role",
        variations=[
            "Tu es un conseiller en achats expérimenté.",
            "En tant qu'expert e-commerce spécialisé dans les recommandations personnalisées :",
            "Agis comme un data scientist spécialisé en systèmes de recommandation.",
            "Tu es un consultant retail avec 10 ans d'expérience."
        ]
    ),
    PromptGene(
        name="task_instruction",
        type="instruction",
        variations=[
            "Recommande 3 produits adaptés au profil client donné.",
            "Analyse le profil client et suggère les 3 meilleurs produits correspondant à ses besoins.",
            "Basé sur les données client, identifie 3 produits qui maximiseront sa satisfaction.",
            "Crée une recommandation personnalisée de 3 produits en analysant les préférences client."
        ]
    ),
    PromptGene(
        name="context_guide",
        type="constraint",
        variations=[
            "Considère le budget, les préférences et l'historique d'achat.",
            "Prends en compte : budget disponible, style personnel, achats précédents, et besoins exprimés.",
            "Analyse : contraintes budgétaires, goûts personnels, saisonnalité, et tendances actuelles.",
            "Évalue budget, préférences, contexte d'usage, et rapport qualité-prix."
        ]
    ),
    PromptGene(
        name="output_format",
        type="format",
        variations=[
            "Présente chaque produit avec nom, prix, et raison de la recommandation.",
            "Pour chaque produit : Nom | Prix | Pourquoi ce choix | Score de pertinence (/10)",
            "Format : 🏷️ [Nom] - [Prix] \n💡 [Justification] \n⭐ [Score de match] /10",
            "Structure : Produit + Prix + Avantages clés + Niveau de recommandation (Excellent/Bon/Correct)"
        ]
    ),
    PromptGene(
        name="quality_constraint",
        type="constraint",
        variations=[
            "Assure-toi que chaque recommandation est pertinente et bien justifiée.",
            "Prioritise la pertinence et la valeur ajoutée pour le client.",
            "Vise l'excellence : chaque recommandation doit être parfaitement adaptée.",
            "Sois précis, pertinent et orienté satisfaction client."
        ]
    )
]

# Contextes de test pour l'optimisation
test_contexts = [
    "Client homme, 30 ans, budget 200€, aime la tech et le sport",
    "Cliente femme, 25 ans, budget 500€, passionnée de mode et voyage", 
    "Client mixte, 45 ans, budget 300€, recherche des cadeaux famille",
    "Cliente femme, 35 ans, budget 150€, écologique et minimaliste"
]

# Lancer l'optimisation
optimizer = GeneticPromptOptimizer(evaluator)

optimization_result = optimizer.optimize(
    template=template,
    genes=optimization_genes,
    test_contexts=test_contexts,
    generations=6,  # Réduire pour la démo
    population_size=12,
    mutation_rate=0.4
)

# Afficher les résultats
print("\n" + "="*70)
print("🧬 RÉSULTATS DE L'OPTIMISATION GÉNÉTIQUE")
print("="*70)

print(f"\n🏆 Meilleure fitness atteinte : {optimization_result['best_fitness']:.3f}")

print(f"\n✨ PROMPT OPTIMISÉ :")
print("="*40)
print(optimization_result['best_prompt'])
print("="*40)

# Détail des gènes du meilleur individu
print(f"\n🧬 Composition génétique du meilleur prompt :")
for gene in optimization_result['best_chromosome'].genes:
    print(f"  {gene.name.upper()}: {gene.current_value}")

# Visualiser l'évolution
print(f"\n📈 Évolution de la fitness sur {len(optimizer.best_fitness_history)} générations")
optimizer.visualize_evolution()

## 4. 📈 Dashboard de Monitoring en Temps Réel

### Système de Monitoring et Analytics

In [ ]:
class PromptMonitoringDashboard:
    """Dashboard de monitoring des performances des prompts"""
    
    def __init__(self, evaluator: PromptEvaluator):
        self.evaluator = evaluator
    
    def generate_performance_report(self, days: int = 7) -> Dict:
        """Génère un rapport de performance sur les N derniers jours"""
        # Récupérer les données
        df = self.evaluator.get_evaluation_history(limit=1000)
        
        if df.empty:
            return {'error': 'Aucune donnée disponible'}
        
        # Convertir timestamp
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        
        # Filtrer par période
        cutoff_date = datetime.now() - timedelta(days=days)
        df_period = df[df['timestamp'] >= cutoff_date]
        
        if df_period.empty:
            return {'error': f'Aucune donnée sur les {days} derniers jours'}
        
        # Calculer les métriques
        report = {
            'period_days': days,
            'total_evaluations': len(df_period),
            'avg_overall_score': df_period['overall_score'].mean(),
            'metrics': {
                'accuracy': {
                    'mean': df_period['accuracy'].mean(),
                    'std': df_period['accuracy'].std(),
                    'trend': self._calculate_trend(df_period, 'accuracy')
                },
                'usefulness': {
                    'mean': df_period['usefulness'].mean(),
                    'std': df_period['usefulness'].std(),
                    'trend': self._calculate_trend(df_period, 'usefulness')
                },
                'coherence': {
                    'mean': df_period['coherence'].mean(),
                    'std': df_period['coherence'].std(),
                    'trend': self._calculate_trend(df_period, 'coherence')
                }
            },
            'performance': {
                'avg_response_time': df_period['response_time'].mean(),
                'total_cost': df_period['cost_usd'].sum(),
                'avg_cost_per_request': df_period['cost_usd'].mean(),
                'cost_efficiency': df_period['overall_score'].sum() / df_period['cost_usd'].sum() if df_period['cost_usd'].sum() > 0 else 0
            },
            'quality_distribution': {
                'excellent': len(df_period[df_period['overall_score'] >= 0.8]),
                'good': len(df_period[(df_period['overall_score'] >= 0.6) & (df_period['overall_score'] < 0.8)]),
                'average': len(df_period[(df_period['overall_score'] >= 0.4) & (df_period['overall_score'] < 0.6)]),
                'poor': len(df_period[df_period['overall_score'] < 0.4])
            }
        }
        
        return report
    
    def _calculate_trend(self, df: pd.DataFrame, metric: str) -> str:
        """Calcule la tendance d'une métrique"""
        if len(df) < 10:
            return "insufficient_data"
        
        # Diviser en deux moitiés
        mid_point = len(df) // 2
        first_half = df.iloc[:mid_point][metric].mean()
        second_half = df.iloc[mid_point:][metric].mean()
        
        change = (second_half - first_half) / first_half if first_half > 0 else 0
        
        if change > 0.05:
            return "increasing"
        elif change < -0.05:
            return "decreasing"
        else:
            return "stable"
    
    def create_dashboard_visualization(self, report: Dict, save_path: str = None):
        """Crée une visualisation dashboard"""
        if 'error' in report:
            print(f"❌ Erreur : {report['error']}")
            return
        
        fig = plt.figure(figsize=(16, 12))
        
        # 1. Métriques principales (2x2 grid en haut)
        gs = fig.add_gridspec(3, 4, height_ratios=[1, 1, 1.2], hspace=0.3, wspace=0.3)
        
        # Score global
        ax1 = fig.add_subplot(gs[0, 0])
        score = report['avg_overall_score']
        color = 'green' if score > 0.7 else 'orange' if score > 0.5 else 'red'
        ax1.pie([score, 1-score], labels=['Score', ''], colors=[color, 'lightgray'], 
               startangle=90, counterclock=False)
        ax1.set_title(f'Score Global\n{score:.1%}', fontweight='bold')
        
        # Nombre d'évaluations
        ax2 = fig.add_subplot(gs[0, 1])
        ax2.bar(['Évaluations'], [report['total_evaluations']], color='steelblue')
        ax2.set_title(f'Total Évaluations\n{report["total_evaluations"]}', fontweight='bold')
        ax2.set_ylabel('Nombre')
        
        # Coût total
        ax3 = fig.add_subplot(gs[0, 2])
        cost = report['performance']['total_cost']
        ax3.bar(['Coût'], [cost], color='coral')
        ax3.set_title(f'Coût Total\n${cost:.3f}', fontweight='bold')
        ax3.set_ylabel('USD')
        
        # Efficience
        ax4 = fig.add_subplot(gs[0, 3])
        efficiency = report['performance']['cost_efficiency']
        ax4.bar(['Efficience'], [efficiency], color='gold')
        ax4.set_title(f'Efficience\n{efficiency:.1f}', fontweight='bold')
        ax4.set_ylabel('Score/USD')
        
        # 2. Métriques détaillées (barres avec tendances)
        ax5 = fig.add_subplot(gs[1, :])
        metrics_names = list(report['metrics'].keys())
        metrics_values = [report['metrics'][m]['mean'] for m in metrics_names]
        metrics_std = [report['metrics'][m]['std'] for m in metrics_names]
        
        bars = ax5.bar(metrics_names, metrics_values, yerr=metrics_std, 
                      capsize=5, color=['skyblue', 'lightgreen', 'plum'], alpha=0.8)
        
        # Ajouter les tendances
        for i, (bar, metric) in enumerate(zip(bars, metrics_names)):
            trend = report['metrics'][metric]['trend']
            trend_symbol = '📈' if trend == 'increasing' else '📉' if trend == 'decreasing' else '➡️'
            ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                    trend_symbol, ha='center', va='bottom', fontsize=16)
        
        ax5.set_title('Métriques de Qualité (avec Tendances)', fontweight='bold', fontsize=14)
        ax5.set_ylabel('Score (0-1)')
        ax5.set_ylim(0, 1)
        
        # 3. Distribution de qualité (camembert)
        ax6 = fig.add_subplot(gs[2, :2])
        quality_dist = report['quality_distribution']
        labels = ['Excellent (≥80%)', 'Bon (60-80%)', 'Moyen (40-60%)', 'Faible (<40%)']
        values = [quality_dist['excellent'], quality_dist['good'], 
                 quality_dist['average'], quality_dist['poor']]
        colors = ['green', 'lightgreen', 'orange', 'red']
        
        wedges, texts, autotexts = ax6.pie(values, labels=labels, colors=colors, 
                                          autopct='%1.1f%%', startangle=90)
        ax6.set_title('Distribution de la Qualité', fontweight='bold', fontsize=14)
        
        # 4. Métriques de performance
        ax7 = fig.add_subplot(gs[2, 2:])
        perf_metrics = ['Temps Réponse (s)', 'Coût Moyen ($)', 'Efficience']
        perf_values = [
            report['performance']['avg_response_time'],
            report['performance']['avg_cost_per_request'],
            report['performance']['cost_efficiency']
        ]
        
        # Normaliser pour visualisation
        normalized_values = [
            min(perf_values[0] / 2, 1),  # Cap à 2s
            min(perf_values[1] / 0.01, 1),  # Cap à $0.01
            min(perf_values[2] / 100, 1)  # Cap à 100
        ]
        
        bars = ax7.barh(perf_metrics, normalized_values, 
                       color=['lightcoral', 'lightsalmon', 'lightseagreen'])
        
        # Ajouter les valeurs réelles
        for bar, value in zip(bars, perf_values):
            ax7.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                    f'{value:.3f}', va='center', fontweight='bold')
        
        ax7.set_title('Métriques de Performance', fontweight='bold', fontsize=14)
        ax7.set_xlim(0, 1.2)
        
        # Titre général
        fig.suptitle(f'Dashboard Monitoring Prompts - {report["period_days"]} derniers jours', 
                    fontsize=16, fontweight='bold')
        
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
        
        plt.show()
        
        return fig
    
    def generate_alert_system(self, report: Dict) -> List[str]:
        """Génère des alertes basées sur les métriques"""
        alerts = []
        
        if 'error' in report:
            return [f"🚫 Erreur système : {report['error']}"]
        
        # Vérifier le score global
        if report['avg_overall_score'] < 0.5:
            alerts.append("🔴 CRITIQUE : Score global très bas (<50%)")
        elif report['avg_overall_score'] < 0.7:
            alerts.append("🟡 ATTENTION : Score global modéré (<70%)")
        
        # Vérifier les tendances
        for metric, data in report['metrics'].items():
            if data['trend'] == 'decreasing':
                alerts.append(f"📉 TENDANCE BAISSIÈRE : {metric} en diminution")
        
        # Vérifier les coûts
        avg_cost = report['performance']['avg_cost_per_request']
        if avg_cost > 0.05:
            alerts.append(f"💰 COÛT ÉLEVÉ : ${avg_cost:.4f} par requête")
        
        # Vérifier la qualité
        poor_ratio = report['quality_distribution']['poor'] / report['total_evaluations']
        if poor_ratio > 0.2:
            alerts.append(f"⚠️ QUALITÉ : {poor_ratio:.1%} de prompts de faible qualité")
        
        # Vérifier l'efficience
        if report['performance']['cost_efficiency'] < 10:
            alerts.append("📊 EFFICIENCE FAIBLE : Ratio qualité/coût sous-optimal")
        
        if not alerts:
            alerts.append("✅ Toutes les métriques sont dans les normes")
        
        return alerts
    
    def export_report(self, report: Dict, filename: str = None) -> str:
        """Exporte le rapport en JSON"""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"prompt_report_{timestamp}.json"
        
        # Ajouter metadata
        export_data = {
            'generated_at': datetime.now().isoformat(),
            'report_type': 'prompt_monitoring',
            'data': report
        }
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(export_data, f, indent=2, ensure_ascii=False, default=str)
        
        return filename

### 🎯 Génération du Dashboard avec données simulées

In [ ]:
# Générer quelques données de test pour le dashboard
print("📊 Génération de données pour le dashboard...")

# Simuler quelques évaluations pour avoir des données
test_prompts = [
    "Résume cet article en 3 points.",
    "Traduis ce texte en anglais professionnel.",
    "Écris un email de suivi commercial.",
    "Crée une liste de contrôle pour projet.",
    "Analyse les risques de cette stratégie."
]

# Évaluer quelques prompts pour créer des données
for i, prompt in enumerate(test_prompts):
    try:
        context = f"Contexte test {i+1}"
        metrics = evaluator.evaluate_prompt_complete(prompt, context)
        print(f"  ✅ Prompt {i+1} évalué (score: {metrics.overall_score():.3f})")
    except Exception as e:
        print(f"  ❌ Erreur prompt {i+1}: {e}")

# Créer le dashboard
dashboard = PromptMonitoringDashboard(evaluator)

# Générer le rapport
print("\n📈 Génération du rapport de performance...")
performance_report = dashboard.generate_performance_report(days=7)

if 'error' not in performance_report:
    # Afficher le rapport textuel
    print("\n" + "="*60)
    print("📊 RAPPORT DE PERFORMANCE - 7 DERNIERS JOURS")
    print("="*60)
    
    print(f"\n📋 Résumé :")
    print(f"  • Total évaluations : {performance_report['total_evaluations']}")
    print(f"  • Score global moyen : {performance_report['avg_overall_score']:.1%}")
    print(f"  • Coût total : ${performance_report['performance']['total_cost']:.4f}")
    print(f"  • Efficience : {performance_report['performance']['cost_efficiency']:.1f}")
    
    print(f"\n📊 Métriques détaillées :")
    for metric, data in performance_report['metrics'].items():
        trend_emoji = {'increasing': '📈', 'decreasing': '📉', 'stable': '➡️'}
        emoji = trend_emoji.get(data['trend'], '❓')
        print(f"  • {metric.capitalize()}: {data['mean']:.3f} (±{data['std']:.3f}) {emoji}")
    
    print(f"\n🎯 Distribution qualité :")
    dist = performance_report['quality_distribution']
    total = performance_report['total_evaluations']
    print(f"  • Excellent: {dist['excellent']} ({dist['excellent']/total:.1%})")
    print(f"  • Bon: {dist['good']} ({dist['good']/total:.1%})")
    print(f"  • Moyen: {dist['average']} ({dist['average']/total:.1%})")
    print(f"  • Faible: {dist['poor']} ({dist['poor']/total:.1%})")
    
    # Générer les alertes
    print(f"\n🚨 Alertes :")
    alerts = dashboard.generate_alert_system(performance_report)
    for alert in alerts:
        print(f"  {alert}")
    
    # Créer la visualisation
    print(f"\n📈 Génération du dashboard visuel...")
    dashboard.create_dashboard_visualization(performance_report)
    
    # Exporter le rapport
    export_filename = dashboard.export_report(performance_report)
    print(f"\n💾 Rapport exporté : {export_filename}")
    
else:
    print(f"❌ Erreur : {performance_report['error']}")
    print("ℹ️  Exécutez d'abord quelques évaluations pour générer des données.")

## 5. 🎯 Synthèse et Bonnes Pratiques

### Framework Complet d'Optimisation

Voici une classe qui combine toutes les techniques vues dans ce notebook :

In [ ]:
class ComprehensivePromptOptimizer:
    """Framework complet d'optimisation de prompts"""
    
    def __init__(self):
        self.evaluator = PromptEvaluator()
        self.ab_tester = PromptABTester(self.evaluator)
        self.genetic_optimizer = GeneticPromptOptimizer(self.evaluator)
        self.dashboard = PromptMonitoringDashboard(self.evaluator)
    
    def comprehensive_optimization_pipeline(self, 
                                          base_prompt: str,
                                          optimization_goal: str,
                                          test_contexts: List[str]) -> Dict:
        """Pipeline complet d'optimisation"""
        
        print("🚀 PIPELINE D'OPTIMISATION COMPLET")
        print("="*50)
        
        results = {
            'base_prompt': base_prompt,
            'goal': optimization_goal,
            'stages': {}
        }
        
        # Stage 1: Évaluation initiale
        print("\n📊 Stage 1: Évaluation Baseline")
        baseline_metrics = self.evaluator.evaluate_prompt_complete(base_prompt)
        results['stages']['baseline'] = {
            'metrics': baseline_metrics,
            'score': baseline_metrics.overall_score()
        }
        print(f"   Score baseline: {baseline_metrics.overall_score():.3f}")
        
        # Stage 2: A/B Testing avec variations manuelles
        print("\n🧪 Stage 2: A/B Testing")
        improved_prompt = self._generate_improved_variant(base_prompt, optimization_goal)
        
        ab_config = ABTestConfig(
            test_name=f"Optimization: {optimization_goal}",
            prompt_a=base_prompt,
            prompt_b=improved_prompt,
            sample_size=15,
            test_contexts=test_contexts
        )
        
        ab_result = self.ab_tester.run_ab_test(ab_config)
        results['stages']['ab_test'] = ab_result
        
        best_ab_prompt = improved_prompt if ab_result.winner == 'B' else base_prompt
        print(f"   Gagnant A/B: {ab_result.winner} (lift: {ab_result.lift_percentage:+.1f}%)")
        
        # Stage 3: Optimisation génétique
        print("\n🧬 Stage 3: Optimisation Génétique")
        
        # Créer un template et des gènes basés sur le meilleur prompt A/B
        genetic_template, genetic_genes = self._create_genetic_components(best_ab_prompt)
        
        genetic_result = self.genetic_optimizer.optimize(
            template=genetic_template,
            genes=genetic_genes,
            test_contexts=test_contexts,
            generations=5,
            population_size=10
        )
        
        results['stages']['genetic'] = genetic_result
        print(f"   Score génétique final: {genetic_result['best_fitness']:.3f}")
        
        # Stage 4: Validation finale
        print("\n✅ Stage 4: Validation Finale")
        final_prompt = genetic_result['best_prompt']
        final_metrics = self.evaluator.evaluate_prompt_complete(final_prompt)
        
        results['stages']['final'] = {
            'prompt': final_prompt,
            'metrics': final_metrics,
            'score': final_metrics.overall_score()
        }
        
        # Calculer l'amélioration totale
        total_improvement = (
            (final_metrics.overall_score() - baseline_metrics.overall_score()) / 
            baseline_metrics.overall_score() * 100
        )
        
        results['summary'] = {
            'total_improvement': total_improvement,
            'baseline_score': baseline_metrics.overall_score(),
            'final_score': final_metrics.overall_score(),
            'final_prompt': final_prompt
        }
        
        print(f"\n🏆 RÉSULTAT FINAL:")
        print(f"   Amélioration totale: {total_improvement:+.1f}%")
        print(f"   Score baseline: {baseline_metrics.overall_score():.3f}")
        print(f"   Score final: {final_metrics.overall_score():.3f}")
        
        return results
    
    def _generate_improved_variant(self, base_prompt: str, goal: str) -> str:
        """Génère une variante améliorée du prompt"""
        improvement_prompt = f"""
        Améliore ce prompt pour mieux atteindre l'objectif spécifié :
        
        PROMPT ORIGINAL : {base_prompt}
        
        OBJECTIF D'AMÉLIORATION : {goal}
        
        Crée une version améliorée qui :
        1. Garde l'intention originale
        2. Optimise pour l'objectif donné
        3. Ajoute de la structure si nécessaire
        4. Améliore la clarté
        
        Réponds uniquement avec le prompt amélioré.
        """
        
        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": improvement_prompt}],
                temperature=0.7
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            print(f"Erreur génération variante : {e}")
            return base_prompt + "\n\nSois précis et détaillé."
    
    def _create_genetic_components(self, prompt: str) -> Tuple[str, List[PromptGene]]:
        """Crée les composants génétiques à partir d'un prompt"""
        # Template simple avec placeholders
        template = "{instruction}\n\n{enhancement}\n\n{format}"
        
        # Gènes génériques pour amélioration
        genes = [
            PromptGene(
                name="instruction",
                type="instruction",
                variations=[
                    prompt,
                    f"En tant qu'expert, {prompt.lower()}",
                    f"Analyse méthodiquement et {prompt.lower()}",
                    f"Étape par étape : {prompt.lower()}"
                ]
            ),
            PromptGene(
                name="enhancement",
                type="constraint",
                variations=[
                    "",
                    "Sois précis et factuel.",
                    "Assure-toi que ta réponse est complète et bien structurée.",
                    "Vise l'excellence et la clarté."
                ]
            ),
            PromptGene(
                name="format",
                type="format",
                variations=[
                    "",
                    "Présente ta réponse de manière claire et organisée.",
                    "Structure ta réponse avec des points principaux.",
                    "Utilise des sections et des exemples si pertinent."
                ]
            )
        ]
        
        return template, genes
    
    def generate_optimization_report(self, results: Dict) -> str:
        """Génère un rapport complet d'optimisation"""
        report = "\n" + "="*80 + "\n"
        report += "🎯 RAPPORT COMPLET D'OPTIMISATION DE PROMPT\n"
        report += "="*80 + "\n"
        
        report += f"\n📋 OBJECTIF : {results['goal']}\n"
        
        report += f"\n📊 RÉSUMÉ DES PERFORMANCES :\n"
        report += f"  • Score baseline      : {results['summary']['baseline_score']:.3f}\n"
        report += f"  • Score final         : {results['summary']['final_score']:.3f}\n"
        report += f"  • Amélioration totale  : {results['summary']['total_improvement']:+.1f}%\n"
        
        report += f"\n🧪 DÉTAIL DES STAGES :\n"
        
        if 'ab_test' in results['stages']:
            ab = results['stages']['ab_test']
            report += f"  📈 A/B Test - Gagnant: {ab.winner} (Lift: {ab.lift_percentage:+.1f}%)\n"
        
        if 'genetic' in results['stages']:
            genetic = results['stages']['genetic']
            report += f"  🧬 Génétique - Score final: {genetic['best_fitness']:.3f}\n"
        
        report += f"\n✨ PROMPT FINAL OPTIMISÉ :\n"
        report += "─" * 40 + "\n"
        report += results['summary']['final_prompt'] + "\n"
        report += "─" * 40 + "\n"
        
        return report

### 🎯 Exemple : Pipeline complet d'optimisation

In [ ]:
# Exemple d'utilisation du pipeline complet
optimizer = ComprehensivePromptOptimizer()

# Prompt de base à optimiser
base_prompt = "Explique ce concept technique."

# Objectif d'optimisation
optimization_goal = "Maximiser la clarté et l'utilité pour des non-experts"

# Contextes de test
test_contexts = [
    "Concept: Intelligence Artificielle",
    "Concept: Blockchain", 
    "Concept: Machine Learning",
    "Concept: Cloud Computing"
]

# Exécuter le pipeline complet
optimization_results = optimizer.comprehensive_optimization_pipeline(
    base_prompt=base_prompt,
    optimization_goal=optimization_goal,
    test_contexts=test_contexts
)

# Générer et afficher le rapport
final_report = optimizer.generate_optimization_report(optimization_results)
print(final_report)

# Visualiser l'évolution génétique si disponible
if 'genetic' in optimization_results['stages']:
    print("\n📈 Évolution génétique :")
    optimizer.genetic_optimizer.visualize_evolution()

## 📚 Bonnes Pratiques et Recommandations

### 🎯 Guide de Sélection des Techniques

| Situation | Technique Recommandée | Raison |
|-----------|----------------------|--------|
| **Prompt unique critique** | A/B Testing simple | Validation statistique rapide |
| **Optimisation continue** | Monitoring Dashboard | Suivi de performance |
| **Exploration créative** | Algorithme Génétique | Découverte de solutions inattendues |
| **Amélioration incrémentale** | Métriques + A/B | Optimisation mesurée |
| **Déploiement production** | Pipeline complet | Robustesse maximale |

### 💡 Conseils d'Implémentation

#### 🔧 Métriques
- **Commencez simple** : Utilisez 3-5 métriques maximum
- **Alignez business** : Vos métriques doivent refléter vos objectifs réels
- **Évitez la sur-optimisation** : Ne pas optimiser pour les métriques mais pour l'usage

#### 🧪 A/B Testing
- **Taille d'échantillon** : Minimum 30 échantillons par variant
- **Significativité** : Attendez p < 0.05 ET effect size > 0.2
- **Contexte varié** : Testez sur plusieurs types d'usage

#### 🧬 Optimisation Génétique
- **Patience** : 10-20 générations minimum
- **Diversité** : Maintenez de la variation dans la population
- **Coût vs Bénéfice** : Technique coûteuse, réservée aux cas critiques

#### 📊 Monitoring
- **Temps réel** : Intégrez dans vos workflows existants
- **Alertes automatiques** : Configurez des seuils d'alerte
- **Tendances** : Surveillez l'évolution plus que les valeurs absolues

### ⚡ Optimisations de Performance

```python
# Bonnes pratiques de performance
best_practices = {
    'batch_evaluation': 'Groupez les évaluations pour réduire les coûts API',
    'caching': 'Cachez les résultats d\'évaluations similaires',
    'parallel_processing': 'Utilisez ThreadPoolExecutor pour paralléliser',
    'smart_sampling': 'Adaptez la taille d\'échantillon selon l\'importance',
    'early_stopping': 'Arrêtez l\'optimisation si convergence atteinte'
}
```

## 🚀 Exercices Pratiques

### Exercice 1 : Créer vos propres métriques métier

Implémentez des métriques spécifiques à votre domaine :

In [ ]:
# Exemple pour l'e-commerce
class ECommercePromptMetrics(PromptMetrics):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Ajoutez vos métriques métier
        self.conversion_potential: float = 0.0
        self.personalization_score: float = 0.0
        self.urgency_factor: float = 0.0
    
    def business_score(self) -> float:
        """Score orienté business pour l'e-commerce"""
        return (
            self.conversion_potential * 0.4 +
            self.personalization_score * 0.3 +
            self.urgency_factor * 0.3
        )

# Votre implémentation ici
# Définissez des métriques pour votre domaine

### Exercice 2 : Système d'alertes intelligent

Créez un système d'alertes qui s'adapte aux patterns de votre usage :

In [ ]:
class AdaptiveAlertSystem:
    def __init__(self):
        self.baseline_metrics = {}
        self.alert_thresholds = {}
    
    def learn_baseline(self, historical_data: pd.DataFrame):
        """Apprend les patterns normaux pour définir les seuils"""
        # Votre implémentation ici
        pass
    
    def generate_smart_alerts(self, current_metrics: Dict) -> List[str]:
        """Génère des alertes adaptées au contexte"""
        # Votre implémentation ici
        pass

# Implémentez votre système d'alertes intelligent

## 🎓 Pour aller plus loin

### 📚 Ressources avancées :
- [A/B Testing Guide for AI Systems](https://example.com)
- [Genetic Algorithms in NLP](https://example.com)
- [MLOps for Prompt Engineering](https://example.com)

### 🔬 Techniques avancées à explorer :
1. **Bayesian Optimization** pour l'optimisation continue
2. **Multi-Armed Bandits** pour l'exploration vs exploitation
3. **Neural Architecture Search** pour l'optimisation de structure
4. **Reinforcement Learning** pour l'apprentissage adaptatif

### 🚀 Intégrations recommandées :
- **MLflow** pour le tracking des expérimentations
- **Weights & Biases** pour le monitoring avancé
- **Apache Airflow** pour l'orchestration des pipelines
- **FastAPI** pour créer des APIs d'optimisation

---

**Prochain notebook** : 04_Systemes_Dynamiques.ipynb - Templates adaptatifs et systèmes auto-améliorés